# Open Food Facts Data Exploration

In [1]:
import pandas as pd
import numpy as np

#### Data Loading

In [2]:
data_folder = 'data/'
data = pd.read_csv(data_folder + 'en.openfoodfacts.org.products.csv', sep='\t')

/home/david/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,3,5,24,25,26,28,36,41,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


We can see the size of the data is not a problem; we can load it easily using pandas.

### Data Preprocessing

#### Choosing The Fields
There are 174 different fields in this OpenFoodFacts dataset. After explorating each field and their value counts, here are the 38 fields we decided to keep (for now). The reasons we dropped the other fields are the following: either
- they were too poorly represented (less than 50 occurrences)
- they were too specific (micronutrients)
- some other field conveyed the same information in a clearer format
- they are of no interest for this study (barcodes, image urls)

In [195]:
chosen_fields = ['product_name', 'generic_name', 'quantity', 'packaging_tags', 'brands_tags',
                 'origins_tags', 'manufacturing_places_tags', 'labels_en', 
                 'purchase_places', 'stores', 'countries_en', 'ingredients_text', 'allergens', 
                 'traces_en', 'additives_n', 'additives_en', 'ingredients_from_palm_oil_n', 
                 'ingredients_that_may_be_from_palm_oil_n', 'nutrition_grade_fr', 'pnns_groups_1', 
                 'main_category_en', 'energy_100g', 'energy-from-fat_100g', 'fat_100g', 
                 'saturated-fat_100g', 'monounsaturated-fat_100g', 'polyunsaturated-fat_100g', 
                 'omega-3-fat_100g', 'omega-6-fat_100g', 'omega-9-fat_100g', 'trans-fat_100g', 
                 'cholesterol_100g', 'carbohydrates_100g', 'sugars_100g', 'proteins_100g', 'sodium_100g', 
                 'nutrition-score-fr_100g', 'nutrition-score-uk_100g']

In [196]:
chosen_data = data[chosen_fields]

#### Data Cleaning

In [197]:
chosen_data['pnns_groups_1'].value_counts().sum()

257415

In [198]:
chosen_data['main_category_en'].value_counts().sum()

180239

In [206]:
chosen_data['brands_tags'].value_counts().sum()

464667

In [209]:
chosen_data['packaging_tags'].value_counts().sum()

140651